#Johan Suarez - 200713010
#Lady Oñate - 200814451


# Exercise 04

# Fraud Detection

## Introduction

- Fraud Detection Dataset from Microsoft Azure: [data](http://gallery.cortanaintelligence.com/Experiment/8e9fe4e03b8b4c65b9ca947c72b8e463)

Fraud detection is one of the earliest industrial applications of data mining and machine learning. Fraud detection is typically handled as a binary classification problem, but the class population is unbalanced because instances of fraud are usually very rare compared to the overall volume of transactions. Moreover, when fraudulent transactions are discovered, the business typically takes measures to block the accounts from transacting to prevent further losses. 

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [3]:
import zipfile
with zipfile.ZipFile('../datasets/fraud_detection.csv.zip', 'r') as z:
    f = z.open('15_fraud_detection.csv')
    data = pd.io.parsers.read_table(f, index_col=0, sep=',')

In [4]:
data.head()

,accountAge,digitalItemCount,sumPurchaseCount1Day,sumPurchaseAmount1Day,sumPurchaseAmount30Day,paymentBillingPostalCode - LogOddsForClass_0,accountPostalCode - LogOddsForClass_0,paymentBillingState - LogOddsForClass_0,accountState - LogOddsForClass_0,paymentInstrumentAgeInAccount,ipState - LogOddsForClass_0,transactionAmount,transactionAmountUSD,ipPostalCode - LogOddsForClass_0,localHour - LogOddsForClass_0,Label
0,2000,0,0,0.00,720.25,5.064533,0.421214,1.312186,0.566395,3279.574306,1.218157,599.00,626.164650,1.259543,4.745402,0
1,62,1,1,1185.44,2530.37,0.538996,0.481838,4.401370,4.500157,61.970139,4.035601,1185.44,1185.440000,3.981118,4.921349,0
2,2000,0,0,0.00,0.00,5.064533,5.096396,3.056357,3.155226,0.000000,3.314186,32.09,32.090000,5.008490,4.742303,0
3,1,1,0,0.00,0.00,5.064533,5.096396,3.331154,3.331239,0.000000,3.529398,133.28,132.729554,1.324925,4.745402,0
4,1,1,0,0.00,132.73,5.412885,0.342945,5.563677,4.086965,0.001389,3.529398,543.66,543.660000,2.693451,4.876771,0


In [5]:
data.shape, data.Label.sum(), data.Label.mean()

((138721, 16), 797, 0.0057453449730033666)

In [6]:
X = data.drop(['Label'], axis=1)
y = data['Label']

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

C:\Users\Andres Piraban Acero\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Exercice 04.1

Estimate a Logistic Regression

Evaluate using the following metrics:
* Accuracy
* F1-Score
* F_Beta-Score (Beta=10)

Comment about the results

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import f1_score

logreg = LogisticRegression(C=1e9)

In [7]:
logreg.fit(X_train, y_train)
y_pred= logreg.predict(X_test)

print('accuracy    ', metrics.accuracy_score(y_test, y_pred))
print('f1_score    ', metrics.f1_score(y_test, y_pred))
print('f1 beta score    ', metrics.fbeta_score(y_test, y_pred,10))


accuracy     0.993973645512
f1_score     0.0
f1 beta score     0.0


C:\Users\Andres Piraban Acero\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


# Exercice 04.2

Under-sample the negative class using random-under-sampling

Which is parameter for target_percentage did you choose?
How the results change?

**Only apply under-sampling to the training set, evaluate using the whole test set**

In [16]:
#Para saber el numero de datos por categoria
pd.crosstab(index=data["Label"],columns="Count")

col_0,Count
Label,
0,137924
1,797


In [17]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)

# Fit and transform x to visualise inside a 2D feature space
x_vis = pca.fit_transform(X)

In [45]:
def UnderSampling(X, y, target_percentage=0.5, seed=None):
    # Assuming minority class is the positive
    n_samples = y.shape[0]
    n_samples_0 = (y == 0).sum()
    n_samples_1 = (y == 1).sum()

    n_samples_0_new =  n_samples_1 / target_percentage - n_samples_1
    n_samples_0_new_per = n_samples_0_new / n_samples_0

    filter_ = y == 0

    np.random.seed(seed)
    rand_1 = np.random.binomial(n=1, p=n_samples_0_new_per, size=n_samples)
    
    filter_ = filter_ & rand_1
    filter_ = filter_ | (y == 1)
    filter_ = filter_.astype(bool)
    
    return X[filter_], y[filter_]

In [46]:
#Invoco la funcion de under sampling para generar el x
x_under,y_under=UnderSampling(X_train,y_train)
pd.crosstab(index=y_under,columns="Count")

col_0,Count
Label,
0,554
1,588


In [28]:
#Entreno el modelo con el undersampling
logreg = LogisticRegression(C=1e9)
logreg.fit(x_under, y_under)
y_pred_under= logreg.predict(X_test)

In [29]:
#Valido el modelo con el test completo
print('accuracy    ', metrics.accuracy_score(y_test, y_pred_under))
print('f1_score    ', metrics.f1_score(y_test, y_pred_under))
print('f1 beta score    ', metrics.fbeta_score(y_test, y_pred_under,10))


accuracy     0.678123468181
f1_score     0.0231031766868
f1 beta score     0.415094339623


# Exercice 04.3

Now using random-over-sampling

In [53]:
import random
def OverSampling(X, y, target_percentage=0.5, seed=None):
    # Assuming minority class is the positive
    n_samples = y.shape[0]
    n_samples_0 = (y == 0).sum()
    n_samples_1 = (y == 1).sum()

    n_samples_1_new =  -target_percentage * n_samples_0 / (target_percentage- 1)

    np.random.seed(seed)
    filter_ = np.random.choice(X[y == 1].shape[0], int(n_samples_1_new))
    # filter_ is within the positives, change to be of all
    filter_ = np.nonzero(y == 1)[0][filter_]
    
    filter_ = np.concatenate((filter_, np.nonzero(y == 0)[0]), axis=0)
    
    return X.iloc[filter_], y.iloc[filter_]

In [54]:
#Invoco la funcion de over sampling para generar el x
x_over, y_over=OverSampling(X_train,y_train)

In [56]:
#Entreno el modelo con el oversampling
logreg = LogisticRegression(C=1e9)
logreg.fit(x_over, y_over)
y_pred_over= logreg.predict(X_test)

In [57]:
#Valido el modelo con el test completo
print('accuracy    ', metrics.accuracy_score(y_test, y_pred_over))
print('f1_score    ', metrics.f1_score(y_test, y_pred_over))
print('f1 beta score    ', metrics.fbeta_score(y_test, y_pred_over,10))

accuracy     0.535624693636
f1_score     0.019601874962
f1 beta score     0.43808933671


# Exercice 04.4*
Evaluate the results using SMOTE

Which parameters did you choose?

In [39]:
def SMOTE(X, y, target_percentage=0.5, k=5, seed=None):
    
    n_samples = y.shape[0]
    n_samples_0 = (y == 0).sum()
    n_samples_1 = (y == 1).sum()
    
    #New samples
    n_samples_1_new =  int(-target_percentage * n_samples_0 / (target_percentage- 1) - n_samples_1)
    
    # A matrix to store the synthetic samples
    new = np.zeros((n_samples_1_new, X.shape[1]))
    
    # Create seeds
    np.random.seed(seed)
    seeds = np.random.randint(1, 1000000, 3)
    
    # Select examples to use as base
    np.random.seed(seeds[0])
    sel_ = np.random.choice(y[y==1].shape[0], n_samples_1_new)
    
    # Define random seeds (2 per example)
    np.random.seed(seeds[1])
    nn__ = np.random.choice(k, n_samples_1_new)
    np.random.seed(seeds[2])
    steps = np.random.uniform(size=n_samples_1_new)  

    # For each selected examples create one synthetic case
    for i, sel in enumerate(sel_):
        # Select neighbor
        nn_ = nn__[i]
        step = steps[i]
        # Create new sample
        new[i, :] = X[y==1].iloc[sel] - step * (X[y==1].iloc[sel] - X[y==1].iloc[nn_])
    
    X = np.vstack((X, new))
    y = np.append(y, np.ones(n_samples_1_new))
    
    return X, y

In [40]:
#Invoco la funcion de over sampling para generar el x
x_smote,y_smote=SMOTE(X_train,y_train)

In [41]:
#Entreno el modelo con el undersampling
logreg = LogisticRegression(C=1e9)
logreg.fit(x_smote, y_smote)
y_pred_smote= logreg.predict(X_test)

In [42]:
#Valido el modelo con el test completo
print('accuracy    ', metrics.accuracy_score(y_test, y_pred_smote))
print('f1_score    ', metrics.f1_score(y_test, y_pred_smote))
print('f1 beta score    ', metrics.fbeta_score(y_test, y_pred_smote,10))

accuracy     0.706669357862
f1_score     0.0143396957659
f1 beta score     0.241003482523


# Exercice 04.5

Estimate a Logistic Regression, GaussianNB, K-nearest neighbors and a Decision Tree **Classifiers**

Evaluate using the following metrics:
* Accuracy
* F1-Score
* F_Beta-Score (Beta=10)

Comment about the results

Combine the classifiers and comment

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [43]:
# fit a logistic regression model and store the class predictions

models = {'lg': LogisticRegression(),
          'dt': DecisionTreeClassifier(),
          'nb': GaussianNB(),
          'nn': KNeighborsClassifier(),
          'rf': RandomForestClassifier()}

for model in models.keys():
    models[model].fit(X, y)
    
# predict test for each model
y_pred = pd.DataFrame(index=data.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X)
    
from sklearn import metrics
from sklearn.metrics import f1_score
for model in models.keys():
    print('accuracy    '+ model, metrics.accuracy_score(y, y_pred[model]))
    print('f1_score    '+ model, metrics.f1_score(y, y_pred[model]))
    print('f1 beta score ' + model, metrics.fbeta_score(y, y_pred[model],10))

accuracy    lg 0.994254655027
f1_score    lg 0.0
f1 beta score lg 0.0
accuracy    dt 0.999762112442
f1_score    dt 0.978859705317
f1 beta score dt 0.958987870352
accuracy    nb 0.993923054188


C:\Users\Andres Piraban Acero\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


f1_score    nb 0.0047225501771
f1 beta score nb 0.0025329153605
accuracy    nn 0.994492542585
f1_score    nn 0.152993348115
f1 beta score nn 0.0873253555542
accuracy    rf 0.998572674649
f1_score    rf 0.859375
f1 beta score rf 0.760854677441


# Exercice 04.6

Using the under-sampled dataset

Evaluate a RandomForestClassifier and compare the results

change n_estimators=100, what happened

In [52]:
# fit a logistic regression model and store the class predictions

modelsrf= RandomForestClassifier()
modelsrf.fit(x_under, y_under)
    
# predict test for each model
y_pred = pd.DataFrame(index=data.index, columns=models.keys())
y_predrf = modelsrf.predict(X_test)
    
from sklearn import metrics
from sklearn.metrics import f1_score

print('accuracy    ', metrics.accuracy_score(y_test, y_predrf))
print('f1_score    ', f1_score(y_test, y_predrf))
print('f1 beta score ', metrics.fbeta_score(y_test, y_predrf,10))

accuracy     0.774516305758
f1_score     0.0324177183865
f1 beta score  0.459840822994
